In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Demandas iniciadas por provincia

In [2]:
# Importo Demandas iniciadas por provincia
Dem_prv=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Demandas iniciadas por provincia.xlsx', sheet_name = 'Demandas iniciadas ', usecols='A:AK')

# Transpongo la información
Dem_prv=pd.melt(Dem_prv, id_vars=['Jurisdicción'], var_name='Periodo', value_name="Demandas_iniciadas")

#Cambio el tipo de dato
Dem_prv['Demandas_iniciadas'] = pd.to_numeric(Dem_prv['Demandas_iniciadas'])

#Filtro periodos
Dem_prv=Dem_prv[Dem_prv['Periodo']<'2019-05-01']

# Redondeo el dato
Dem_prv['Demandas_iniciadas']=round(Dem_prv['Demandas_iniciadas'])

In [3]:
# EXPORTO DEMANDAS POR PROVINCIA

Dem_prv.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\Dt_Demandas_Iniciadas_UART.xlsx', sheet_name='Dt_Demandas_iniciadas', index = False)

## Demandas Iniciadas Anuales

In [4]:
# Anualizo el dato (2017- Actualidad)

# Creo la variable año
Dem_prv['Año'] = pd.DatetimeIndex(Dem_prv['Periodo']).year

# Agrupo por Año
Dem_prv_agrup=Dem_prv.groupby(['Año'], as_index=False)["Demandas_iniciadas"].sum()

# Importo Información anual (2010-2016)
Dem_prv1=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Demandas iniciadas anual.xlsx')

# Junto ambas tablas
Dem_prv_anual=Dem_prv1.append(Dem_prv_agrup)

In [5]:
# EXPORTO STOCK ANUAL DEMANDAS

Dem_prv_anual.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\Dt_Demandas_Iniciadas_anual.xlsx', sheet_name='Dt_Demandas_anual', index = False)


## Juicios Cerrados (RENALI)

In [6]:
# Importo los datasets Sentencias de Primera Instancia (RENALI)

Prim_inst_4T2017=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\4-2017 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')
Prim_inst_1T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\1-2018 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')
Prim_inst_2T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\2-2018 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')
Prim_inst_3T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\3-2018 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')
Prim_inst_4T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\4-2018 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')
Prim_inst_1T2019=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\1-2019 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')

In [7]:
# Junto los datasets

PRIM_INST=Prim_inst_4T2017.append(Prim_inst_1T2018)
PRIM_INST=PRIM_INST.append(Prim_inst_2T2018)
PRIM_INST=PRIM_INST.append(Prim_inst_3T2018)
PRIM_INST=PRIM_INST.append(Prim_inst_4T2018)
PRIM_INST=PRIM_INST.append(Prim_inst_1T2019)


In [8]:
# Importo dataset de compañías para tener el cod_cia y la denominación corta

# OJO QUE AÚN NO APARECE MUTUAL PETROLEROS ART, CUANDO APAREZCA HAY QUE AGREGARLA AL DATASET EN "ASEGURADORA"

Dataset_compañías=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Compañías.xlsx', sheet_name='Sentencias')

PRIM_INST=PRIM_INST.merge(Dataset_compañías, how='inner', on='Aseguradora')

PRIM_INST=PRIM_INST.drop('Aseguradora',1)

PRIM_INST['Dataset']='Dataset_1ra_instancia'

In [9]:
# Creo variable Fecha
PRIM_INST['Fecha']=PRIM_INST['F_1ra_sent']

# Creo variable Monto
PRIM_INST['Monto']=PRIM_INST['MONTO PRIMERA INST']

# Creo variable instancia
PRIM_INST['Instancia']=PRIM_INST['Tipo sentencia 1° inst.']

# Filtro y me quedo son con los registros que necesito
PRIM_INST_filt=PRIM_INST[(PRIM_INST['Instancia']=='Homologación.')|(PRIM_INST['Instancia']=='Hace lugar a la demanda totalmente.')|(PRIM_INST['Instancia']=='Hace lugar a la demanda parcialmente.')]

# Reemplazo
PRIM_INST_filt['Instancia']=['Demandas conciliadas' if PRIM_INST_filt['Tipo sentencia 1° inst.'][i]=='Homologación.' else 'Sentencia 1° instancia' for i in PRIM_INST_filt.index]

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
# Listo las etiquetas de las columnas

cols = list(PRIM_INST_filt.columns.values)
cols

['N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Tipo sentencia 1° inst.',
 'F_1ra_sent',
 'MONTO PRIMERA INST',
 '% PRIMERA INST',
 'Tipo sentencia 2° inst.',
 'f_2da_sent',
 'MONTO SEGUNDA INST',
 '% SEGUNDA INST',
 'cod_cia',
 'denominacion_corta',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

In [11]:
# Reordeno las columnas

reord= [ 'cod_cia',
 'denominacion_corta',
 'N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

PRIM_INST_filt=PRIM_INST_filt[reord]

In [12]:
#EXPORTO SENTENCIAS DE PRIMER INSTANCIA

# PRIM_INST.to_excel(r'C:\Users\eangenello\Desktop\TÉCNICA\TABLERO ART\Datasets\Dt_Sentencias_Primera_Instancia.xlsx', sheet_name='Dt_Sent_Primera_Inst', index=False)

In [13]:
# Importo los datasets Sentencias de Primera Instancia (RENALI)

Seg_inst_4T2017=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\4-2017 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')
Seg_inst_1T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\1-2018 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')
Seg_inst_2T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\2-2018 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')
Seg_inst_3T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\3-2018 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')
Seg_inst_4T2018=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\4-2018 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')
Seg_inst_1T2019=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\1-2019 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')

In [14]:
# Junto los datasets

SEG_INST=Seg_inst_4T2017.append(Seg_inst_1T2018)
SEG_INST=SEG_INST.append(Seg_inst_2T2018)
SEG_INST=SEG_INST.append(Seg_inst_3T2018)
SEG_INST=SEG_INST.append(Seg_inst_4T2018)
SEG_INST=SEG_INST.append(Seg_inst_1T2019)

In [15]:
SEG_INST=SEG_INST.merge(Dataset_compañías, how='inner', on='Aseguradora')

SEG_INST=SEG_INST.drop('Aseguradora',1)

SEG_INST['Dataset']='Dataset_2da_instancia'

In [16]:
# Creo variable Fecha
SEG_INST['Fecha']=SEG_INST['f_2da_sent']

# Creo variable Monto
SEG_INST['Monto']=SEG_INST['MONTO SEGUNDA INST']

# Creo variable instancia
SEG_INST['Instancia']=SEG_INST['Tipo sentencia 2° inst.']

# Filtro y me quedo son con los registros que necesito
SEG_INST_filt=SEG_INST[(SEG_INST['Instancia']=='Homologación.')|(SEG_INST['Instancia']=='Hace lugar a la demanda totalmente.')|(SEG_INST['Instancia']=='Hace lugar a la demanda parcialmente.')]

SEG_INST_filt['Instancia']='Sentencia 2° instancia'

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [17]:
# Listo las etiquetas de las columnas

cols = list(SEG_INST_filt.columns.values)
cols

['N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Tipo sentencia 1° inst.',
 'F_1ra_sent',
 'MONTO PRIMERA INST',
 '% PRIMERA INST',
 'Tipo sentencia 2° inst.',
 'f_2da_sent',
 'MONTO SEGUNDA INST',
 '% SEGUNDA INST',
 'cod_cia',
 'denominacion_corta',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

In [18]:
# Reordeno las columnas

reord= [ 'cod_cia',
 'denominacion_corta',
 'N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

SEG_INST_filt=SEG_INST_filt[reord]

In [19]:
# Junto Primera y Segunda Instancia

DT_Demandas_Instancias=PRIM_INST_filt.append(SEG_INST_filt)

In [20]:
DT_Demandas_Instancias['year']= pd.DatetimeIndex(DT_Demandas_Instancias['Fecha']).year
DT_Demandas_Instancias['month']= pd.DatetimeIndex(DT_Demandas_Instancias['Fecha']).month

In [21]:
# Defino la función que va a asignar el trimestre a partir de la variable mes

def trim(row):
    if DT_Demandas_Instancias.month.iloc[row]==1 or DT_Demandas_Instancias.month.iloc[row]==2 or DT_Demandas_Instancias.month.iloc[row]==3:
        return 1
    
    elif DT_Demandas_Instancias.month.iloc[row]==4 or DT_Demandas_Instancias.month.iloc[row]==5 or DT_Demandas_Instancias.month.iloc[row]==6:
        return 2
    
    elif DT_Demandas_Instancias.month.iloc[row]==7 or DT_Demandas_Instancias.month.iloc[row]==8 or DT_Demandas_Instancias.month.iloc[row]==9:
        return 3
    
    else:
        return 4

In [22]:
# Creo la variable Trimestre y le aplico la función definida

DT_Demandas_Instancias['trimestre']=[trim(i) for i in range(len(DT_Demandas_Instancias.index))]

In [23]:
# Creo la variable Periodo Balance a partir de las varibles year y trimestre

DT_Demandas_Instancias['Periodo_balance'] = DT_Demandas_Instancias.year.astype(str).str.cat(DT_Demandas_Instancias.trimestre.astype(str), sep='-')

In [24]:
DT_Demandas_Instancias=DT_Demandas_Instancias.drop(['year','month','trimestre'],1)

In [25]:
#EXPORTO 

DT_Demandas_Instancias.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\Dt_Demandas_Instancias.xlsx', sheet_name='Dt_Demandas_Instancias', index=False)